In [1]:
import os
import openai
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 생성 (최신 방식)
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate

template_text = "안녕하세요. 제 이름은 {name}이고 나이는 {age}입니다."

In [3]:
llm = ChatOpenAI()
result = llm.invoke("오늘 저녁 메뉴 추천 해줘")

print(result)

content='오늘의 저녁 메뉴로는 스파게티나 파스타, 그리고 샐러드와 빵으로 구성된 이탈리안 코스요리를 추천드립니다. 또는 바비큐 그릴로 구운 스테이크나 그릴드 치킨과 채소를 곁들인 아메리칸 스타일의 요리도 좋을 것 같습니다. 간단하게 만들 수 있는 집밥으로는 된장찌개나 불고기 등 한국의 전통 음식도 좋은 선택이 될 것입니다. 요리를 즐기시는데 맛있는 시간 되시길 바랍니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 24, 'total_tokens': 224, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7040b375-3b68-4b5a-b0df-bca13c1dde74-0' usage_metadata={'input_tokens': 24, 'output_tokens': 200, 'total_tokens': 224, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
llm = ChatOpenAI(model='gpt-4o-mini')
result = llm.invoke("오늘 저녁 메뉴 추천 해줘")

print(result)

In [ ]:
import os
import csv
import openai
from dotenv import load_dotenv

# .env 파일 로드 (API 키 보안을 위해)
load_dotenv()

# OpenAI API 클라이언트 생성
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 모델별 토큰당 가격 설정 (최신 가격 확인 필요)
price_per_1k_prompt = 0.15  # 예시 가격 (1K 토큰당 $0.15)
price_per_1k_completion = 0.60  # 예시 가격 (1K 토큰당 $0.60)

def log_cost(response):
    """응답에서 토큰 사용량을 추출하고 비용을 CSV에 저장"""
    usage = response.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens
    total_tokens = usage.total_tokens

    # 비용 계산
    prompt_cost = (prompt_tokens / 1000) * price_per_1k_prompt
    completion_cost = (completion_tokens / 1000) * price_per_1k_completion
    total_cost = prompt_cost + completion_cost

    # CSV 파일에 저장
    with open("llm_cost_log.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([prompt_tokens, completion_tokens, total_tokens, prompt_cost, completion_cost, total_cost])

    print(f"Logged cost: ${total_cost:.5f}")

# OpenAI API 호출
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Explain quantum computing"}]
)

# 비용 로깅
log_cost(response)


In [ ]:
prompt = ChatPromptTemplate.from_template("{input}의 위치와 수도는?")

chain = prompt | llm
result = chain.invoke({"input":"대한민국"})

print(result.content)

output_parser는 LLM의 응답의 정제 및 후처리를 자동화 해주는 툴이다.

In [ ]:
prompt = ChatPromptTemplate.from_template("{input}의 위치와 수도는?")

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
result = chain.invoke({"input":"북한"})

print(result)

새로운 chain 연결하기

In [8]:
prompt = ChatPromptTemplate.from_template("{input}의 위치와 수도는?")

output_parser = StrOutputParser()

chain1 = prompt | llm | output_parser

prompt_translate = ChatPromptTemplate.from_template("{word}를 영어로 직역해줘")

chain2 = {"word":chain1} | prompt_translate | llm | output_parser

result1 = chain1.invoke({"input":"북한"})
result2 = chain2.invoke({"input":"북한"})

print("result1:",result1)
print("result2:",result2)


result1: 북한은 한반도 북쪽에 위치하고 있으며, 수도는 평양입니다.
result2: North Korea is a country located in East Asia, north of Seoul. The capital is Pyongyang.
